# Часть 1. Предобработка текста и классификация

In [ ]:
import warnings
warnings.filterwarnings('ignore')

### Прочитайте текст в файлах positive.txt и negative.txt

In [ ]:
with open("positive.txt", "r", encoding="utf-8") as file:
    positive_plain = file.read()

with open("negative.txt", "r", encoding="utf-8") as file:
    negative_plain = file.read()

In [ ]:
print(positive_plain[:400], negative_plain[:400], sep="\n\n")

### Разбейте данные на предложения

Для токенизации используйте nltk.sent_tokenize. чтобы удалить имен пользователей из сообщений напишите регулярное выражение.

<b>Классы символов в регулярных выражениях</b>:

[A-Z] – символы верхнего регистра (латиница)

[a-z] – символы нижнего регистра (латиница)

[А-Я] – символы верхнего регистра (кириллица)

[а-я] – символы нижнего регистра (кириллица)

[0-9] или \d – цифра

[^0-9] или \D – любой символ, кроме цифры

. – Один любой символ, кроме новой строки \n.

? – 0 или 1 вхождение шаблона слева

\+ – 1 и более вхождений шаблона слева

\* – 0 и более вхождений шаблона слева

\w – Любая цифра или буква (\W — все, кроме буквы или цифры)

\d – Любая цифра [0-9] (\D — все, кроме цифры)

\s – Любой пробельный символ (\S — любой непробельнй символ)

\b – Граница слова

[..] – дин из символов в скобках ([^..] — любой символ, кроме тех, что в скобках)

\ – Экранирование специальных символов (\. означает точку или \+ — знак «плюс»)

^ и $ – Начало и конец строки соответственно

{n,m} – От n до m вхождений ({,m} — от 0 до m)

a|b – Соответствует a или b

() – Группирует выражение и возвращает найденный текст

\t, \n, \r – Символ табуляции, новой строки и возврата каретки соответственно

In [ ]:
import re
from nltk import sent_tokenize


def split_data(text):
    # Избавляемся от имен пользователей, указанных в письмах
    name = re.compile(<your code here>)
    res = name.sub("", text)
    # Удалите лишние переносы строк и разбейте на предложения по знаку "."
    res = <your code here>
    return res

In [ ]:
positive = split_data(positive_plain)
negative = split_data(negative_plain)

In [ ]:
len(positive), len(negative)

In [ ]:
pos_labels = [1] * len(positive)

In [ ]:
neg_labels = [0] * len(negative)

Объединим все в один список

In [ ]:
all_text = positive + negative
all_labels = pos_labels + neg_labels

### Предварительный анализ коллекции

#### Средняя длина предложений

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(all_text)

In [ ]:
len_data = df[0].apply(len)

In [ ]:
len_data.describe()

#### Длины текстов в символах

In [ ]:
import matplotlib.pyplot as plt

%pylab inline

In [ ]:
fig, ax = plt.subplots()

n, bins, patches = ax.hist(len_data.tolist())

fig.show()

#### Самые частые слова

Токенизируйте предложения на слова (nltk.word_tokenize)

In [ ]:
from nltk import word_tokenize


all_text = [word_tokenize(line) for line in all_text]

In [ ]:
from nltk import FreqDist


n_types = []
n_tokens = []
fd = FreqDist()
for line in all_text:
    fd.update(line)
    n_types.append(len(fd))
    n_tokens.append(sum(list(fd.values())))
for i in fd.most_common(10):
    print(i)

#### Закон Ципфа

В любом достаточно большом тексте ранг типа обратно пропорционален его частоте: f=a/r

f – частота типа, r – ранг типа, a – параметр, для славянских языков – около 0.07

In [ ]:
freqs = list(fd.values())
freqs = sorted(freqs, reverse = True)

fig, ax = plt.subplots()
ax.plot(freqs[:300], range(300))
plt.show()

#### Закон Хипса

С увеличением длины текста (количества токенов), количество типов увеличивается в соответствии с законом: |V|=K∗N^b

N – число токенов, |V| – количество типов в словаре, K,b – параметры, обычно K∈[10,100],b∈[0.4,0.6]

In [ ]:
fig, ax = plt.subplots()
ax.plot(n_types, n_tokens)
plt.show()

### Подготовим данные для nltk.NaiveBayesClassifier

Классификатор принимает данные о предложении в виде словаря {"слово": #количество встреч в предожении}

Используйсте Counter. Пример работы:

Counter('abracadabra')

>Counter({'a': 5, 'b': 2, 'c': 1, 'd': 1, 'r': 2})

In [ ]:
from collections import Counter

In [ ]:
all_text = <your code here>

### Обучим модель

In [ ]:
from sklearn.metrics import accuracy_score
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_text, all_labels, test_size=0.1)

In [ ]:
clf = NaiveBayesClassifier.train(zip(X_train, y_train))

### Протестируем качество классификации

In [ ]:
pred = clf.classify_many(X_test)
accuracy_score(y_test, pred)

# Добавим признаков

Для каждого примера добавим количество частей речи в нем. Если мы встретили знак пунктуации, то обозначим его как 'PNCT'.

In [ ]:
import pymorphy2

In [ ]:
morph = pymorphy2.analyzer.MorphAnalyzer()

Используйте pymorphy2.analyzer.MorphAnalyzer.parse

In [ ]:
all_text_with_pos = []
for sample in all_text:
    update = Counter(sample)
    for word, count in sample.items():
        <your code here>
    all_text_with_pos.append(update)

In [ ]:
print(all_text_with_pos[0])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_text_with_pos, all_labels, test_size=0.1)

In [ ]:
clf = NaiveBayesClassifier.train(zip(X_train, y_train))

In [ ]:
pred = clf.classify_many(X_test)
accuracy_score(y_test, pred)

### Попробуем лемматизировать слова

Обучите тот же классификатор но на примерах, где все слова в нормальной форме.

In [ ]:
all_text_with_pos = []
for sample in all_text:
    update = Counter()
    for word, count in sample.items():
        <your code here>
    all_text_with_pos.append(update)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(all_text_with_pos, all_labels, test_size=0.1)
clf = NaiveBayesClassifier.train(zip(X_train, y_train))

pred = clf.classify_many(X_test)
accuracy_score(y_test, pred)

### Как изменилось качество? Почему?

# Часть 2. Использование знаний морфологии для генерации текста

Задание: написать шаблон вида: {person} {action} {count} {target}. Где

Person - лицо которое выполняет действие action. Каждое такое действие может быть в трех временах (настоящее, прошедшее и будущее). Действие выполняется с целью target. Такая цель - это некоторый объект или объекты числом count. count>0.

Используйте make_agree_with_number, parse и inflect из библиотеки pymorphy2.

Времена глаголов в документации pymorphy2: past (прошедшее), pres (настоящее), futr (будущее).

In [ ]:
import pymorphy2
morph = pymorphy2.analyzer.MorphAnalyzer()

Согласование существительного с числом.

In [ ]:
butyavka = morph.parse("бутявка")[0]
butyavka.make_agree_with_number(5)

Чтобы поставить слово в нужную форму используется inflect

In [ ]:
butyavka.inflect({'gent'})  # нет кого? (родительный падеж)

In [ ]:
butyavka.inflect({'plur', 'gent'})

In [ ]:
def genarate(person, action, time, count, target):
    # Поставьте action в нужное время
    action = <your code here>
    # Согласуйте target с числом
    target = <your code here>
    return "{person} {action} {count} {target}".format(person=person, action=action, count=count, target=target)

In [ ]:
genarate("Лиза", "съела", "past", 2, "груша")

# Часть 3. Статистические морфологические анализаторы

### Загрузим данные из nltk.treebank

In [ ]:
from nltk.corpus import treebank

In [ ]:
sentences = treebank.tagged_sents()

In [ ]:
print(sentences[0])

### Выделим признаки

In [ ]:
def features(sentence, index):
    return {
        'word': sentence[index],
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1]
        # Придумайте еще признаков
        # <your code here>
    }

In [ ]:
def untag(tagged_sentence):
    return [w for w, t in tagged_sentence]

In [ ]:
# Split the dataset for training and testing
cutoff = int(.75 * len(sentences))
training_sentences = sentences[:cutoff]
test_sentences = sentences[cutoff:]


def transform_to_dataset(tagged_sentences):
    X, y = [], []
 
    for tagged in tagged_sentences:
        for index in range(len(tagged)):
            X.append(features(untag(tagged), index))
            y.append(tagged[index][1])
 
    return X, y
 
X, y = transform_to_dataset(training_sentences)

### В качестве классификатора используем DecisionTreeClassifier

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline


clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', DecisionTreeClassifier(criterion='entropy'))
])


# Используем не все примеры (может не хватить оперативной памяти или долго обучаться)
clf.fit(X[:10000], y[:10000])

In [ ]:
X_test, y_test = transform_to_dataset(test_sentences)

In [ ]:
clf.score(X_test[:100], y_test[:100])

### Классифицируем с помощью CRF

In [ ]:
train, test = sentences[:-100], sentences[-100:]

In [ ]:
from nltk.tag import CRFTagger

In [ ]:
ct = CRFTagger()

In [ ]:
ct.train(train ,'model.crf.tagger')

In [ ]:
ct.evaluate(test)